In [4]:
import pandas as pd 
import numpy as np
import copy

#data = pd.read_csv("train.csv") 
#ran = np.random.rand(len(data)) < 0.8
#train = data[ran]
#val = data[~ran]
#train.to_csv('UsedTrainingData.csv')
#val.to_csv('UsedValidationData.csv')

train =  pd.read_csv('UsedTrainingData.csv')
val = pd.read_csv('UsedValidationData.csv')

targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
examples = list(train.index.values) 

NumericalAttributes = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']
CategoricalAttributes = ['Work_accident', 'promotion_last_5years', 'sales', 'salary']

def getImpurityOfExamples(examplesForCurNode,method):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurNode:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurNode)
    PropOfNeg = NumberOfNegEx/len(examplesForCurNode)
    if method == 'Gini':
        return (2*PropOfPos*PropOfNeg)
    elif method == 'misClasifi':
        return min(PropOfPos,PropOfNeg)
    elif method == 'Entropy':
        if PropOfPos == 0:
            return (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))
        elif PropOfNeg == 0:
            return (-1*PropOfPos*(np.log(PropOfPos)/np.log(2)))
        else:
            return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

def getMethodImpurity(Attribute,value,examplesForCurAttr,method): 
    fra1 = train.loc[examplesForCurAttr,:]
    LessThanExamples = []
    MoreThanExamples = []
    if Attribute in NumericalAttributes: 
        for j in examplesForCurAttr:
            if  float(str(fra1.at[int(j),Attribute])) <= float(str(value)):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
    else:
        for j in examplesForCurAttr:
            if  str(fra1.at[int(j),Attribute]) == str(value):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
        
    if len(LessThanExamples) <=0 or len(MoreThanExamples) <= 0:    
        return 999
    else:  
        return ((len(LessThanExamples)*getImpurityOfExamples(LessThanExamples,method))+(len(MoreThanExamples)*getImpurityOfExamples(MoreThanExamples,method)))/len(examplesForCurAttr)
    
class Node(object):
    def __init__(self,examples,decisionAttributeType,decisionAttribute,decisionAttributeValue,level):
        self.examples = examples
        self.decisionAttributeType = decisionAttributeType
        self.decisionAttribute = decisionAttribute        
        self.decisionAttributeValue = decisionAttributeValue
        self.level = level
        self.leftChild = None
        self.rightChild = None
        
    def __str__(self): 
        return "Decision attribute type is "+self.decisionAttributeType+"decisionAttribute is "+ self.decisionAttribute+" decisionAttributeValue is "+ str(self.decisionAttributeValue)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))
    
def BestOfAttributes(examplesForCurAttr,targetAttribute,method):
    fra1 = train.loc[examplesForCurAttr,CategoricalAttributes]
    dictForCategAttrUniqueValues = {}
    for i in CategoricalAttributes:
        dictForCategAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueCateg = copy.deepcopy(dictForCategAttrUniqueValues)
    print("calculating Impurities please wait")
    for i in CategoricalAttributes:
        for j,val in enumerate(dictForCategAttrUniqueValues[i]):
            dictForEntropyUniqueValueCateg[i][j] = getMethodImpurity(i,dictForCategAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForCategoricalAttributes = {}
    for i in CategoricalAttributes:
        EntropyValuesForCategoricalAttributes[i]=min(dictForEntropyUniqueValueCateg[i])
    BestOfCategoricalAttributes = min(EntropyValuesForCategoricalAttributes, key=EntropyValuesForCategoricalAttributes.get)
    BestOfCategoricalAttributesValue = min(EntropyValuesForCategoricalAttributes.values())   
    
    fra1 = train.loc[examplesForCurAttr,NumericalAttributes]
    dictForNumAttrUniqueValues = {}
    for i in NumericalAttributes:
        dictForNumAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueNum = copy.deepcopy(dictForNumAttrUniqueValues)
    
    print("calculating Impurities please wait")
    for i in NumericalAttributes:
        for j,val in enumerate(dictForNumAttrUniqueValues[i]):
            dictForEntropyUniqueValueNum[i][j] = getMethodImpurity(i,dictForNumAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForNumericalAttributes = {}
    for i in NumericalAttributes:
        EntropyValuesForNumericalAttributes[i]=min(dictForEntropyUniqueValueNum[i])
    BestOfNumericalAttributes = min(EntropyValuesForNumericalAttributes, key=EntropyValuesForNumericalAttributes.get)
    BestOfNumericalAttributesValue = min(EntropyValuesForNumericalAttributes.values())
    
    for index, item in enumerate(dictForEntropyUniqueValueNum[BestOfNumericalAttributes]):
        if item == BestOfNumericalAttributesValue:
            BestSplitValueNumerical = dictForNumAttrUniqueValues[BestOfNumericalAttributes][index]
            break
            
    for index, item in enumerate(dictForEntropyUniqueValueCateg[BestOfCategoricalAttributes]):
        if item == BestOfCategoricalAttributesValue:
            BestSplitValueCategorical = dictForCategAttrUniqueValues[BestOfCategoricalAttributes][index]
            break

    if BestOfNumericalAttributesValue < BestOfCategoricalAttributesValue:
        return BestOfNumericalAttributes,BestSplitValueNumerical
    else: 
        return BestOfCategoricalAttributes,BestSplitValueCategorical
    
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,level,method):
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:        
        r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        print(r)
        print("Type1")
        return r
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
        print(r)
        print("Type2")
        return r
    if getImpurityOfExamples(examplesForCurNode,method) < 0.001 or len(examplesForCurNode)<10:
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.5:
            r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
            #numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            #numberOfNodes  = numberOfNodes + 1
        print(r)
        print("Type3")
        return r
    
    BestAttribute,BestAttributeValue = BestOfAttributes(examplesForCurNode,targetAttribute,method)
    
    if BestAttribute in NumericalAttributes:
        BestAttributeType = 'numerical'
    else:
        BestAttributeType = 'categorical'
    r = Node(examplesForCurNode,BestAttributeType,BestAttribute,BestAttributeValue,level)
    print(r)
    print("Type4")
    #numberOfNodes  = numberOfNodes + 1
    examplesForLeftChildOfCurrentNode = []
    examplesForRightChildOfCurrentNode = []        
    if BestAttributeType == 'categorical':        
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(BestAttributeValue):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    else:
        for iter1 in examplesForCurNode:
            if  float(str(train.at[int(iter1),str(BestAttribute)])) <= float(str(BestAttributeValue)):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    r.leftChild = DecisionTreeBuilder(examplesForLeftChildOfCurrentNode,targetAttribute,level+1,method)
    r.rightChild = DecisionTreeBuilder(examplesForRightChildOfCurrentNode,targetAttribute,level+1,method)
    
    return r 


In [5]:
examplesForRootNode = examples[:]
method = 'Entropy'
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,0,method)

calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.46 of level 0 number of examples of this node 8989
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.11 of level 1 number of examples of this node 2501
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 2 number of examples of this node 545
Type1
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 2 of level 2 number of examples of this node 1956
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttri

calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 6 of level 3 number of examples of this node 916
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 287 of level 4 number of examples of this node 908
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 131 of level 5 number of examples of this node 904
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 6 number of examples of this node 116
Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.3 of level 6 number of 

Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 257 of level 14 number of examples of this node 41
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 15 number of examples of this node 5
Type3
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 279 of level 15 number of examples of this node 36
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 16 number of examples of this node 29
Type2
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 16 number of examples of this node 7
Type3
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decision

Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 180 of level 6 number of examples of this node 251
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 7 number of examples of this node 114
Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.48 of level 7 number of examples of this node 137
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 8 number of examples of this node 3
Type1
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is salary decisionAttributeValue is low of level 8 number of examples of this node 134
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribu

Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 264 of level 9 number of examples of this node 76
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.45 of level 10 number of examples of this node 69
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 11 number of examples of this node 1
Type1
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 3 of level 11 number of examples of this node 68
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.61 of level 12 number of examples of this node 23
Type4
Decision at

calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 274 of level 11 number of examples of this node 602
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 12 number of examples of this node 581
Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.89 of level 12 number of examples of this node 21
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 13 number of examples of this node 18
Type2
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 13 number of examples of this node 3
Type3
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisi

calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.55 of level 7 number of examples of this node 387
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is technical of level 8 number of examples of this node 57
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.54 of level 9 number of examples of this node 11
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.81 of level 10 number of examples of this node 10
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 11 number of exam

calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 128 of level 9 number of examples of this node 13
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 10 number of examples of this node 3
Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is marketing of level 10 number of examples of this node 10
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 11 number of examples of this node 2
Type2
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 11 number of examples of this node 8
Type3
calculating Impurities please wait
calculating Impurities please wait
Decision attribute t

calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 5 of level 7 number of examples of this node 514
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.92 of level 8 number of examples of this node 510
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 276 of level 9 number of examples of this node 509
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 238 of level 10 number of examples of this node 507
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttri

In [2]:
def RecursiveFunctionToPredict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decisionAttributeValue == 'Positive':
            return '1'
        else:
            return '0'
    else:
        if r.decisionAttributeType == 'categorical':            
            AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])            
            if AttrValue == str(r.decisionAttributeValue):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)            
        elif r.decisionAttributeType == 'numerical':            
            AttrValue = float(str(TestSet.at[int(i),str(r.decisionAttribute)]))            
            if AttrValue <= float(str(r.decisionAttributeValue)):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)
def predict(RootNode,NameOfTestFile):
    GivenTestData = pd.read_csv(str(NameOfTestFile))
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    return GivenTestData            

In [ ]:
Prediction = predict(RootNode,'sample_test.csv')
Prediction.head()   

In [8]:
    GivenTestData =  train.copy()
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Training Data")
    print("Using Entropy as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(train)

Results On Training Data
Using Entropy as impurity measure
-------------------------------------------------
True Positive are    2082
True Negatives are   6811
False Positive are   25
False Negatives are  71
Precsion is          0.9881347887992407
Recall is            0.9670227589410125
F1_Score is          4.0922190201729105
Accuracy is          0.989320280342641
-------------------------------------------------
( Here 1's  are positive 0's are negative)


8989

In [9]:
    GivenTestData =  val.copy()
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Validation Data")
    print("Using Entropy as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(val)

Results On Validation Data
Using Entropy as impurity measure
-------------------------------------------------
True Positive are    496
True Negatives are   1701
False Positive are   26
False Negatives are  26
Precsion is          0.9501915708812261
Recall is            0.9501915708812261
F1_Score is          4.209677419354839
Accuracy is          0.976878612716763
-------------------------------------------------
( Here 1's  are positive 0's are negative)


2249